In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys

In [12]:
def build_model(width, height, depth, classes):
    inputShape = (height, width, depth) # входной размер
    chanDim = -1
    model = Sequential([ # собираем последовательную модель
        
        # конволюционные слои
        Conv2D(16, (3, 3), padding="same", input_shape=inputShape),
        Activation("relu"),
        BatchNormalization(axis=chanDim),
        MaxPooling2D(pool_size=(2, 2)),

         # конволюционные слои
        Conv2D(32, (3, 3), padding="same"),
        Activation("relu"),
        BatchNormalization(axis=chanDim),
        Conv2D(32, (3, 3), padding="same"),
        Activation("relu"),
        BatchNormalization(axis=chanDim),
        MaxPooling2D(pool_size=(2, 2)),

         # полносвязные слои
        Flatten(),
        Dense(256),
        Activation("relu"),
        BatchNormalization(),
        Dropout(0.5),

        Dense(classes),
        Activation("softmax")
    ])
    return model

In [13]:
def step(X, y):
    # Функция для реализации обучения
    with tf.GradientTape() as tape:
        pred = model(X) # делаем предсказание
        loss = categorical_crossentropy(y, pred) # вычисляем функцию ошибок
    # calculate the gradients using our tape and then update the
    # model weights
    grads = tape.gradient(loss, model.trainable_variables) # расчитываем градиенты обратным распространением
    opt.apply_gradients(zip(grads, model.trainable_variables)) # обновляем веса модели

In [8]:
# initialize the number of epochs to train for, batch size, and
# initial learning rate
EPOCHS = 5 # кол-во эпох
BS = 64 # размер батча
INIT_LR = 1e-3 # скорость обчучения

((trainX, trainY), (testX, testY)) = mnist.load_data() # загружаем данные

# приводим к виду для подачи в модель
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

trainY = to_categorical(trainY, 10) # целевые признаки из категориальных в численные
testY = to_categorical(testY, 10)

[INFO] loading MNIST dataset...


In [9]:
model = build_model(28, 28, 1, 10) # строим модель
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS) # создаем оптимизатор

[INFO] creating model...


In [10]:
numUpdates = int(trainX.shape[0] / BS) # кол-во обновления парамтров за эпоху

for epoch in range(0, EPOCHS): # итерируемся по всем эпохам
    print("starting epoch {}/{}...".format(
        epoch + 1, EPOCHS), end="")
    sys.stdout.flush()
    epochStart = time.time()
    for i in range(0, numUpdates):
        # определяем начальный и конечный индексы
        start = i * BS
        end = start + BS
        step(trainX[start:end], trainY[start:end]) # делаем шаг обучения
    epochEnd = time.time()
    elapsed = (epochEnd - epochStart) / 60.0 # время на эпоху
    print("took {:.4} minutes".format(elapsed)) 


[INFO] starting epoch 1/5...took 1.163 minutes
[INFO] starting epoch 2/5...took 1.437 minutes
[INFO] starting epoch 3/5...took 1.242 minutes
[INFO] starting epoch 4/5...took 1.275 minutes
[INFO] starting epoch 5/5...took 1.3 minutes


In [11]:
# in order to calculate accuracy using Keras' functions we first need
# to compile the model
model.compile(optimizer=opt, loss=categorical_crossentropy, metrics=["acc"]) # компилируем для расчета точности с использованием Keras 

(loss, acc) = model.evaluate(testX, testY) # считаем точность модели
print("[INFO] test accuracy: {:.4f}".format(acc))

313/313 [==============================] - 2s 7ms/step - loss: 0.0376 - acc: 0.9886
[INFO] test accuracy: 0.9886
